In [1]:
import torch
from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, AutoModelForSeq2SeqLMWithValueHead, create_reference_model
from transformers import AutoTokenizer, BartForConditionalGeneration
from trl.core import respond_to_batch, old_respond_to_batch
from tqdm import tqdm
from datasets import Dataset

# 設定訓練迴圈的迭代次數
num_iterations = 100
ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
# model = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
# model_ref = create_reference_model(model)
# 初始化模型和分詞器
# model = BartForConditionalGeneration.from_pretrained('lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans')
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(ar_checkpoint)
# model = AutoModelForCausalLMWithValueHead.from_pretrained('lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans')
model_ref = create_reference_model(model)
tokenizer = AutoTokenizer.from_pretrained('lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans')
tokenizer.pad_token = tokenizer.eos_token

2024-05-22 19:23:22.654465: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-22 19:23:22.705361: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 19:23:24.061150: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [3]:
from datetime import datetime
import os

now = datetime.now()
ts = now.strftime("%m%d-%H%M")
print("timestamp:", ts)
log_dir = f"logs/{ts}"
os.makedirs(log_dir, exist_ok=True)

ppo_config = PPOConfig(batch_size=1, mini_batch_size=1, log_with='tensorboard', learning_rate = 0.1, project_kwargs={'logging_dir': log_dir})
ppo_trainer = PPOTrainer(config = ppo_config, model = model, ref_model=model_ref, tokenizer=tokenizer)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


timestamp: 0522-1923


/work/b0990106x/trl/trl/trainer/ppo_trainer.py:262: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [4]:
for iteration in tqdm(range(100)):
    query_txt = "v_tok_410v_tok_411v_tok_595"
    query_tensor = tokenizer.encode(query_txt, return_tensors="pt")
    query_tensor = query_tensor.to(device)
    response_tensor = old_respond_to_batch(model, query_tensor, txt_len=1000)
    # # Get model response
    # response_tensor = model.generate(query_tensor)
    response_text = tokenizer.decode(response_tensor[0], skip_special_tokens=True)
    # Mimic batch structure
    batch = {
        "query": query_tensor,
        "response": response_text
    }

    reward_length = len(tokenizer.decode(response_tensor[0], skip_special_tokens=True))
    reward_size = response_tensor[0].size(0)
    print("Reward Length:", reward_length)
    print("Reward Size:", reward_size)
    reward_int = 500 - reward_size
    reward = torch.tensor([float(reward_int)], device=device)
    # Print information

    # PPO training step
    train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], [reward])
    
    # Log statistics
    ppo_trainer.log_stats(train_stats, batch, reward)
    
    # Print information
    print(f"Iteration {iteration + 1}, Reward: {reward.item()}, Predicted Text: {response_text}")





  0%|          | 0/100 [00:00<?, ?it/s]You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Reward Length: 453
Reward Size: 57


/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1285: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  std_scores = data["scores"].std()
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1312: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1315: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  stats["tokens/responses_len_std"] = torch.std(response_lens).cpu().num

Iteration 1, Reward: 443.0, Predicted Text: v_tok_410v_tok_410v_tok_865v_tok_922v_tok_865v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62v_tok_62
Reward Length: 9016
Reward Size: 1000


/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (125938.48) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (6067381.00) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (6076727.50) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
  2%|▏         | 2/100 [00:35<33:01, 20.22s/it]

...

Iteration 11, Reward: -500.0, Predicted Text: v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646

 12%|█▏        | 12/100 [05:48<46:06, 31.44s/it]

Iteration 12, Reward: -500.0, Predicted Text: v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646

 13%|█▎        | 13/100 [06:19<45:25, 31.33s/it]

Iteration 13, Reward: -500.0, Predicted Text: v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646v_tok_646

In [ ]:
def get_reward(predicted_list, finish):
    reward = 0
    # predicted_list will be one text of v_tok_410v_tok_411v_tok_595 ...
    # predicted_token will be a list of v_tok_410, v_tok_411, v_tok_595 ... should also contain v_tok
    predicted_token = [f'v_tok_{u}' for u in predicted_list.split("v_tok_")[1:]]
    
    # predicted_list.split("v_tok_")[1:]
    return predicted_token


#encode_input = tokenizer(
            # "".join([f"v_tok_{u + layer_i * 1024}" for u in dataset[f"tgt_encodec_{layer_i}"][0]]),
a = get_reward("v_tok_410v_tok_411v_tok_595", True)
print(a)

['v_tok_410', 'v_tok_411', 'v_tok_595']
